This notebook debugs records development.

20250716

In [ ]:
import straxion
import matplotlib.pyplot as plt
import numpy as np

# Apply the style
plt.style.use('../../xenon.mplstyle')

st = straxion.qualiphide()
st.set_config(
    dict(
        daq_input_dir="timeS429", 
        iq_finescan_dir="finescan_iq428"
    )
)

st.is_stored(
    "timeS429", # run id
    "raw_records", # data type
)

In [ ]:
r = st.get_array(
    "timeS429", # run id
    "records", # data type
)[0] # Only read the first record, as the rest are the same.

In [ ]:
import sys

# Get size of object r in bytes and convert to MB
size_mb = sys.getsizeof(r) / (1024 * 1024)
print(f"Size of record object: {size_mb:.2f} MB")


In [ ]:
slice_sample_length = 20_000
total_sample_length = r["length"]

for i in range(int(total_sample_length / slice_sample_length)):
    sample_range = (i * slice_sample_length, (i + 1) * slice_sample_length)

    times_start = r["time"]
    times_us = (
        r['time'] + np.arange(r['length']) * r['dt'] - times_start
    ) / 1e3
    times_us = times_us[sample_range[0]:sample_range[1]]

    plt.figure(figsize=(10, 5))
    plt.plot(
        times_us, 
        r['data_theta'][sample_range[0]:sample_range[1]], 
        label="Flipped and Shifted", 
        color="black",
        alpha=0.5,
        lw=1
    )
    plt.plot(
        times_us, 
        r['data_theta_moving_average'][sample_range[0]:sample_range[1]], 
        label="Moving Averaged", 
        color="tab:blue",
        lw=2
    )
    plt.plot(
        times_us, 
        r['data_theta_convolved'][sample_range[0]:sample_range[1]], 
        label="Convolved with Pulse Kernel", 
        color="tab:orange",
        lw=2
    )

    plt.legend()
    plt.xlabel("Time Since Run Start [us]")
    plt.ylabel("Theta [rad]")
    plt.title(f"Slice {i+1} of {int(total_sample_length / slice_sample_length)}")
    plt.show()

The few sample offset should be explained by t0

# Details

## Performance of `circfit`

In [5]:

from straxion.plugins.records import PulseProcessing

finescan = PulseProcessing.load_finescan_files("finescan_iq428")
finescan_i = finescan[0][:, 1]
finescan_q = finescan[0][:, 2]
x_center, y_center, radius, residuals = PulseProcessing.circfit(finescan_i, finescan_q)


In [ ]:
plt.scatter(finescan_i, finescan_q, label="finescan_iq428", color="black")
# Create points for circle
theta = np.linspace(0, 2*np.pi, 100)
circle_x = x_center + radius * np.cos(theta)
circle_y = y_center + radius * np.sin(theta)
plt.plot(circle_x, circle_y, 'r-', label='Fitted circle', color="tab:blue")
plt.xlabel("I")
plt.ylabel("Q")
plt.legend(loc='upper left', bbox_to_anchor=(0, 1.2))
#plt.title("Check of the circle fit by iq428.txt")
plt.show()

We might consider using more physical model to fit the circle.


## Pulse kernel

In [ ]:
r["dt"]

In [ ]:
emg_strax = PulseProcessing.pulse_kernel(
    ns=512, 
    fs=50_000, 
    t0=200_000, 
    tau=600_000, 
    sigma=28_000, 
    truncation_factor=10
)
emg_matlab = PulseProcessing.pulse_kernel(
    ns=512, 
    fs=100_000, # dt = 10_000 ns = 10 us
    t0=100_000, 
    tau=300_000, 
    sigma=14_000, 
    truncation_factor=10
)

plt.plot(np.flip(emg_matlab), lw=0.5, label="Recasted from Matlab Result")
plt.plot(np.flip(emg_strax), lw=0.5, label="Straxion default parameters")
plt.xlim(0, 600)
plt.ylim(0, 0.03)
plt.legend()
plt.xlabel("Samples")